In [2]:
import sys
import pickle
import numpy as np
import progress
from progress.bar import IncrementalBar
from keras.models import load_model
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2023-12-11 03:03:16.819748: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 03:03:17.822868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 03:03:19.021169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


No GPU found


2023-12-11 03:03:21.711684: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
thresholds = [0.20, 0.25, 0.30, 0.35, 0.40, 0.45,0.50, 0.55, 0.60]

# Loading the test set and the model 
f = open("X_test.pkl", "rb")
X_test = pickle.load(f)
f.close()

model = load_model("model.h5")

# Computing the predictions for every theta 
predictions = []
bar = IncrementalBar('Processing', max=9)
max_pooling_window = 0
for threshold in thresholds:
    predictions_theta = np.zeros((len(X_test), 11))
    for (key,val) in X_test.items():
        # Predicting on the subset
        val = np.reshape(val, (val.shape[0], val.shape[1], val.shape[2], 1))
        with tf.device('/cpu:0'):
            prediction = model.predict(val)
        
        # Class-wise max pooling:
        if max_pooling_window:
            max_pooled_pred = np.zeros_like(prediction)
            N = len(prediction)
            for i in range(N):
                    max_pooled_pred[i] = np.max(
                            prediction[max(0,i-max_pooling_window//2):
                                min(N,i + max_pooling_window//2), :], axis = 0)
            prediction = max_pooled_pred
        
        # Aggregating the results from the same excerpt
        prediction = np.sum(prediction, axis = 0)
        m = np.max(prediction)
        prediction /= m
        prediction = prediction - threshold
        prediction[prediction > 0] = 1
        prediction[prediction <= 0] = 0
        predictions_theta[key] = prediction
    predictions.append(predictions_theta)
    bar.next()
bar.finish()

# Saving the predictions 
pickle.dump(predictions, open("predictions", "wb"))

1/1 [==============================] - 0s 37ms/step
